# Выбор локации для скважины


Имеются пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. 

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import cross_val_score
from random import randint

In [2]:
data_1 = pd.read_csv('/datasets/geo_data_0.csv')
data_2 = pd.read_csv('/datasets/geo_data_1.csv')
data_3 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
print(data_1.head())

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647


In [4]:
print(data_2.head())

      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305


In [5]:
print(data_3.head())

      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746


In [6]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
data_1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [8]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [9]:
data_2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [10]:
data_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [11]:
data_3.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


In [12]:
data_1 = data_1.drop(['id'], axis = 1)
data_2 = data_2.drop(['id'], axis = 1)
data_3 = data_3.drop(['id'], axis = 1)

### Вывод
Представленные данные были изучены, был удален столбец id, т.к. не содержал значительной информации о скважине и мог снизить качество модели. 

## Обучение и проверка модели

In [13]:
def model(data):
    features = data.drop('product', axis = 1)
    target = data['product']
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)
    pd.options.mode.chained_assignment = None
    numeric = ['f0', 'f1', 'f2']
    scaler = StandardScaler()
    scaler.fit(features_train[numeric])
    features_train[numeric] = scaler.transform(features_train[numeric])
    features_valid[numeric] = scaler.transform(features_valid[numeric]) 
    model = LinearRegression(normalize = False)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    rmse = mean_squared_error(target_valid, predictions_valid)**0.5
    reserves = predictions_valid.mean()
    df = {'target':target_valid, 'predictions':predictions_valid}
    df_frame = pd.DataFrame(df, columns = ['target', 'predictions'])
    return df_frame, rmse, reserves

In [14]:
df_1, rmse_1, reserves_1 = model(data_1)
print('Регион 1')
print('Значение параметра RMSE: {:.3f}'.format(rmse_1))
print('Среднее значение запасов для одной скважины: {:.2f}, тыс. баррелей'.format(reserves_1))

Регион 1
Значение параметра RMSE: 37.579
Среднее значение запасов для одной скважины: 92.59, тыс. баррелей


In [15]:
df_2, rmse_2, reserves_2 = model(data_2)
print('Регион 2')
print('Значение параметра RMSE: {:.3f}'.format(rmse_2))
print('Среднее значение запасов для одной скважины: {:.2f}, тыс. баррелей'.format(reserves_2))

Регион 2
Значение параметра RMSE: 0.893
Среднее значение запасов для одной скважины: 68.73, тыс. баррелей


In [16]:
df_3, rmse_3, reserves_3 = model(data_3)
print('Регион 3')
print('Значение параметра RMSE: {:.3f}'.format(rmse_3))
print('Среднее значение запасов для одной скважины: {:.2f}, тыс. баррелей'.format(reserves_3))

Регион 3
Значение параметра RMSE: 40.030
Среднее значение запасов для одной скважины: 94.97, тыс. баррелей


### Вывод
Для всех данных была использована модель линейной регрессии. Предварительно признаки были масштабированы методом стандартизации. По результатам предсказания можно сделать вывод о том, что в 1 и 3 регионах средние запасы выше, но при этом средняя квадратичная ошибка намного выше, чем в случае 2 региона, поэтому на данном этапе сложно выбрать лучший регион.

## Подготовка к расчёту прибыли

In [17]:
costs = 10e+09  
income = 450000
wells = 200
def profit_estimation(data, reserves):
    big_well = sorted(data['predictions'])[-200:]
    break_even = costs/(income*wells)
    between = break_even - reserves
    print('Объем сырья для безубыточной разработки скважины: {:.2f}, тыс. баррелей'.format(break_even))
    print('Величина средних запасов, превышающих средние: {:.2f}, тыс. баррелей'.format(between))

In [18]:
print('Регион 1')
profit_estimation(df_1, reserves_1)

Регион 1
Объем сырья для безубыточной разработки скважины: 111.11, тыс. баррелей
Величина средних запасов, превышающих средние: 18.52, тыс. баррелей


In [19]:
print('Регион 2')
profit_estimation(df_2, reserves_2)

Регион 2
Объем сырья для безубыточной разработки скважины: 111.11, тыс. баррелей
Величина средних запасов, превышающих средние: 42.38, тыс. баррелей


In [20]:
print('Регион 3')
profit_estimation(df_3, reserves_3)

Регион 3
Объем сырья для безубыточной разработки скважины: 111.11, тыс. баррелей
Величина средних запасов, превышающих средние: 16.15, тыс. баррелей


### Вывод
По исходным данным было получено значение запасов для безубыточной разработки одной скважины, которое составило 111 тыс. баррелей. По средним значениям для всех скважин ни один из регионов не достиг данного значения, и опять же видно, что средние запасы 2 региона ниже запасов 1 и 3.

## Расчёт прибыли и рисков 

In [21]:
def revenue(predictions_valid, target_valid, count):
    probs_sorted = predictions_valid.sort_values(ascending=False)
    selected = target_valid[probs_sorted.index][:count]
    return (sum(selected)*income - costs)

In [22]:
print('Регион 1')
revenue_1 = revenue(df_1['predictions'], df_1['target'], wells)
print('Прибыль при максимальных значениях запасов:', int(revenue_1), 'руб.')

Регион 1
Прибыль при максимальных значениях запасов: 3320826043 руб.


In [23]:
print('Регион 2')
revenue_2 = revenue(df_2['predictions'], df_2['target'], wells)
print('Прибыль при максимальных значениях запасов:', int(revenue_2), 'руб.')

Регион 2
Прибыль при максимальных значениях запасов: 2415086696 руб.


In [24]:
print('Регион 3')
revenue_3 = revenue(df_3['predictions'], df_3['target'], wells)
print('Прибыль при максимальных значениях запасов:', int(revenue_3), 'руб.')

Регион 3
Прибыль при максимальных значениях запасов: 2710349963 руб.


In [25]:
state = np.random.RandomState(12345)
def bootstramp(target,predictions):
    values = []
    minus = 0
    for i in range(1000):
        target_subsample = target.sample(n = 500, replace = True, random_state = state)   
        predictions_subsmaple = predictions[target_subsample.index]
        profit = revenue(predictions_subsmaple, target_subsample, wells)
        values.append(profit)
        if profit < 0:
            minus+=1
    values = pd.Series(values)
    mean_profit = values.mean()
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    print('Средняя прибыль:', int(mean_profit), 'руб.')
    print('Процент риска: {:.1%}'.format(minus/1000))
    print('Доверительный интервал прибыли:', (int(lower), int(upper)), 'руб.')

In [26]:
print('Регион 1')
bootstramp(df_1['target'],df_1['predictions'])

Регион 1
Средняя прибыль: 425938526 руб.
Процент риска: 6.0%
Доверительный интервал прибыли: (-102090094, 947976353) руб.


In [27]:
print('Регион 2')
bootstramp(df_2['target'], df_2['predictions'])

Регион 2
Средняя прибыль: 518259493 руб.
Процент риска: 0.3%
Доверительный интервал прибыли: (128123231, 953612982) руб.


In [28]:
print('Регион 3')
bootstramp(df_3['target'],df_3['predictions'])

Регион 3
Средняя прибыль: 420194005 руб.
Процент риска: 6.2%
Доверительный интервал прибыли: (-115852609, 989629939) руб.


### Вывод
По итогам расчета прибыли для максимальных значений запасов по скважинам, составляющую 3,3 млрд для первого, 2,4 млрд для второго, 2,7 млрд для третьего регионов. По данным значениям можно сделать вывод о том, что предварительно лучшим для разработки окажется первый регион, но т.к. не всегда удастся разрабатывать наилучшие по запасам скважины, необходимо составить распределение прибыли и выбирать по его среднему значению. По итогам расчета распределения прибыли можно сделать вывод о том, что максимальная средняя прибыль во 2 регионе и составляет 518,2 млн и примерно на 100 млн превосходит прибыль для первого и третьего регионов. Также, по нашему условию максимальный риск должен составлять 2,5%, что не соответсвует риску первого и третьего региона, составляющему 6 и 6,3 процента. В то время как риск для второго региона составляет всего 0,3%. Таким образом для разработки скважин предложен второй регион.